In [5]:
import json
import time
import tracemalloc
import math
import heapq
from collections import defaultdict, deque
import networkx as nx

# Carregar dados das cidades
with open("cities.json", "r") as f:
    cities_data = json.load(f)

# Pré-processar: converter população e limitar número de cidades para testes variados
def preprocess_cities(cities_data, n):
    cities = []
    for city in cities_data[:n]:
        cities.append({
            "name": city["city"],
            "latitude": city["latitude"],
            "longitude": city["longitude"],
            "population": int(city["population"].replace(",", ""))
        })
    return cities

# Distância Euclidiana
def euclidean_distance(c1, c2):
    return math.sqrt((c1["latitude"] - c2["latitude"])**2 + (c1["longitude"] - c2["longitude"])**2)

# Construção de grafo para networkx
def build_graph_networkx(cities, r):
    G = nx.Graph()
    for city in cities:
        G.add_node(city["name"], pos=(city["longitude"], city["latitude"]), population=city["population"])
    for i in range(len(cities)):
        for j in range(i + 1, len(cities)):
            dist = euclidean_distance(cities[i], cities[j])
            if dist <= r:
                G.add_edge(cities[i]["name"], cities[j]["name"], weight=dist)
    return G

# Construção de grafo com defaultdict
def build_graph_dict(cities, r):
    graph = defaultdict(list)
    for c1 in cities:
        for c2 in cities:
            if c1 != c2:
                dist = euclidean_distance(c1, c2)
                if dist <= r:
                    graph[c1["name"]].append((c2["name"], dist))
    return graph

# Dijkstra usando networkx
def run_dijkstra_networkx(cities, r, start, goal):
    G = build_graph_networkx(cities, r)
    return nx.dijkstra_path(G, start, goal, weight='weight')

# Busca bidirecional com defaultdict
def bidirectional_search(graph, start, goal):
    if start == goal:
        return [start]

    front_start = {start}
    front_goal = {goal}
    pred_start = {start: None}
    pred_goal = {goal: None}

    while front_start and front_goal:
        next_front = set()
        for u in front_start:
            for v, _ in graph[u]:
                if v not in pred_start:
                    pred_start[v] = u
                    next_front.add(v)
                if v in pred_goal:
                    return reconstruct_path(pred_start, pred_goal, v)
        front_start = next_front

        next_front = set()
        for u in front_goal:
            for v, _ in graph[u]:
                if v not in pred_goal:
                    pred_goal[v] = u
                    next_front.add(v)
                if v in pred_start:
                    return reconstruct_path(pred_start, pred_goal, v)
        front_goal = next_front
    return None

def reconstruct_path(pred_start, pred_goal, meet_point):
    path = []
    node = meet_point
    while node:
        path.append(node)
        node = pred_start[node]
    path.reverse()
    node = pred_goal[meet_point]
    while node:
        path.append(node)
        node = pred_goal[node]
    return path

# Medição de desempenho
def measure_algorithm(algorithm_fn, *args):
    tracemalloc.start()
    start_time = time.time()
    result = algorithm_fn(*args)
    end_time = time.time()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    return {
        "tempo (s)": round(end_time - start_time, 4),
        "memória pico (KB)": round(peak / 1024, 2),
        "nós no caminho": len(result) if result else 0
    }

# Avaliação com 50 cidades e raio 10
sample_size = 50
radius = 10
cities = preprocess_cities(cities_data, sample_size)
start_city = cities[0]["name"]
goal_city = cities[-1]["name"]

# Executar avaliações
resultado_dijkstra = measure_algorithm(run_dijkstra_networkx, cities, radius, start_city, goal_city)
grafo_dict = build_graph_dict(cities, radius)
resultado_bidirecional = measure_algorithm(bidirectional_search, grafo_dict, start_city, goal_city)

import pandas as pd
df = pd.DataFrame([resultado_dijkstra, resultado_bidirecional], index=["Dijkstra (networkx)", "Busca Bidirecional"])
df



,tempo (s),memória pico (KB),nós no caminho
Dijkstra (networkx),0.0025,97.68,5
Busca Bidirecional,0.0000,3.20,4
